# OPTIMIZANDO CODIGOS PARA ALTA PERFORMANCE

## TIPOS DEFINIDOS EM FUNÇÕES

Os tipos são muito importantes em JULIA. Um tipo mal definido pode atrasar e muito o desempenho de uma função/codigo. A melhor forma de obter um bom desempenho é casar o tipo do dado a ser trabalhado com o tipo que a função vai calcular.
No exemplo abaixo, temos uma função que soma elementos de um vetor. No primeiro teste a função soma_vetor1 recebe um vetor de qualquer tipo e trabalha com "soma" do tipo Float (soma = 0.0). No segundo teste a mesma função e vetores, porém a variável soma é do tipo inteiro.

In [56]:
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetor1(x)
    soma = 0.0
    for i in x
        soma = soma + i
    end
    return soma
end

soma_vetor1 (generic function with 1 method)

In [57]:
x1 = collect(0:0.0001:5000);    # Array{Float64,1},
x2 = linspace(0,5000,50000001); # LinSpace{Float64}
x3 = 0:0.0001:5000;             # FloatRange{Float64}

0.0:0.0001:5000.0

In [58]:
# confirmandoo tipo

typeof(x1) , typeof(x2) , typeof(x3)

(Array{Float64,1},LinSpace{Float64},FloatRange{Float64})

In [59]:
@timev soma_vetor1(x1)

  0.297723 seconds (2.55 k allocations: 131.336 KB)
elapsed time (ns): 297722554
bytes allocated:   134488
pool allocs:       2553


1.2500000249999998e11

In [33]:
@timev soma_vetor1(x2)

  0.354286 seconds (16.99 k allocations: 704.616 KB)
elapsed time (ns): 354285647
bytes allocated:   721527
pool allocs:       16986


1.2500000249999998e11

In [34]:
@timev soma_vetor1(x3)

  0.293451 seconds (4.39 k allocations: 228.679 KB)
elapsed time (ns): 293450822
bytes allocated:   234167
pool allocs:       4386


1.2500000249999998e11

O tipo Array{Float64,1} apresentou melhor desempenho de tempo.

A função agora modificada. Note que "soma" agora é do tipo inteiro, mas está recebendo um vetor do tipo Float

In [60]:
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetor2(x)
    soma = 0             # so esse datalhe ja é suficiente para reduzir o desempenho
    for i in x
        soma = soma + i
    end
    return soma
end

soma_vetor2 (generic function with 2 methods)

In [61]:
@timev soma_vetor2(x1)

  1.279769 seconds (100.00 M allocations: 1.490 GB, 9.16% gc time)
elapsed time (ns): 1279768908
gc time (ns):      117212909
bytes allocated:   1600086940
pool allocs:       100001754
GC pauses:         69
full collections:  1


1.2500000249999998e11

In [37]:
@timev soma_vetor2(x2)

  1.456858 seconds (100.00 M allocations: 1.490 GB, 4.94% gc time)
elapsed time (ns): 1456858450
gc time (ns):      71930058
bytes allocated:   1600116925
pool allocs:       100002350
GC pauses:         70


1.2500000249999998e11

In [38]:
@timev soma_vetor2(x3)

  1.358280 seconds (100.00 M allocations: 1.490 GB, 5.30% gc time)
elapsed time (ns): 1358280210
gc time (ns):      72039978
bytes allocated:   1600107936
pool allocs:       100002141
GC pauses:         70


1.2500000249999998e11

## FUNÇÕES ANÔNIMAS

Funções anônimas - funções sem nome - são funções no qual não há nome para defini-las. Um bom exemplo são as funções o **map()** e em compreensões de lista. Não precisam usar o SymPy ou a forma de função genérica (funcao(variavel) = expressao). O uso principal para funções anônimas é passá-los para funções que assumem outras funções como argumentos. Em geral são mais lentas que as funções genéricas ou simbólicas. 

In [55]:
using FastAnonymous

In [7]:
@timev map((w -> w^2),rand(10000,10000));

 19.561381 seconds (200.10 M allocations: 4.475 GB, 1.82% gc time)
elapsed time (ns): 19561381075
gc time (ns):      356042823
bytes allocated:   4805360126
pool allocs:       200104522
non-pool GC allocs:15
malloc() calls:    2
GC pauses:         141
full collections:  1


A mesma função so que agora usando a macro @anon do pacote FastAnonymous

In [28]:
@timev map(@anon(w -> w^2),rand(10000,10000));

  0.618874 seconds (15.83 k allocations: 1.491 GB, 0.97% gc time)
elapsed time (ns): 618873767
gc time (ns):      5986321
bytes allocated:   1600704471
pool allocs:       15825
malloc() calls:    2
GC pauses:         2


## NÃO USE VARIÁVEIS GLOBAIS EM FUNÇÕES

Variáveis globais são uma tentação na vida de qualquer programador. Porém...

In [62]:
# função calcula a soma a partir de uma variável global X
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetorG()
    soma = 0.0             # cuidado! soma é tipo float, se não piora mais ainda
    for i in x
        soma = soma +  i
    end
    return soma
end

soma_vetorG (generic function with 1 method)

In [63]:
x = collect(0:0.0001:5000);

In [64]:
@timev soma_vetorG()

  6.622657 seconds (200.00 M allocations: 3.725 GB, 5.92% gc time)
elapsed time (ns): 6622656677
gc time (ns):      391774147
bytes allocated:   4000044034
pool allocs:       200000432
GC pauses:         174
full collections:  1


1.2500000249999998e11

A mesma função só que usando variável local passada para função

In [65]:
# função calcula a soma a partir de uma variável local X passada como valor
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetorL(x)
    soma = 0.0             # cuidado! soma é tipo float
    for i in x
        soma = soma + i
    end
    return soma
end

soma_vetorL (generic function with 1 method)

In [66]:
x = collect(0:0.0001:5000);

In [67]:
@timev soma_vetorL(x)

  0.066364 seconds (1.62 k allocations: 81.462 KB)
elapsed time (ns): 66363656
bytes allocated:   83417
pool allocs:       1621


1.2500000249999998e11

Diferença absurda de tempo de alocação de memória entre o primeiro caso e segundo caso. 

## UTILIZE FUNÇÕES PARA SUBSTITUIR BLOCOS DE CÓDIGO

Bloco de código que utiliza variáveis globais para calcular a soma todos os elementos de um vetor

In [76]:
tic()
x = collect(0:0.0001:5000)
soma = 0.0             # cuidado! soma é tipo float
for i in x
    soma = soma + i
end

display(soma)

toc()

1.2500000249999998e11

elapsed time: 6.822481982 seconds


6.822481982

Função que substitui o procedimento acima e usa variáveis locais

In [73]:
x = collect(0:0.0001:5000)

# função calcula a soma a partir de uma variável local X passada como valor
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetorL(x)
    soma = 0.0             # cuidado! soma é tipo float
    for i in x
        soma = soma + i
    end
    return soma
end

soma_vetorL (generic function with 1 method)

In [74]:
@timev soma_vetorL(x)

  0.173144 seconds (1.62 k allocations: 81.462 KB)
elapsed time (ns): 173143904
bytes allocated:   83417
pool allocs:       1621


1.2500000249999998e11

Veja que a diferença de tempo é absurda!

## QUAL A MELHOR: FUNÇÕES GENÉRICAS, SIMBÓLICAS OU ANÔNIMAS?